In [18]:
#このセルでは、ひとまずSuperconのデータを使ってcrabnetを適当に試してみる

In [19]:
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [20]:
#まずsuperconのデータを適当に分析する

SC_data = pd.read_csv("/workspaces/mat2vec_codespace/main_code/20240322_Supercon_data_allstr_handedited.csv", encoding='utf-8')
missing_values = SC_data.isnull().sum()
print(missing_values)

data number                                                         0
reference number                                                    0
comment                                                         18281
common formula of materials                                      1518
chemical formula                                                    0
element name of materials                                           0
composition of MA1                                                  0
element name of materials.1                                       439
composition of MA2                                                454
element name of materials.2                                      6699
composition of MA3                                               6716
element name of materials.3                                     17522
composition of MA4                                              17554
element name of materials.4                                     27666
composition of MA5  

In [21]:
crab_input_data = SC_data.loc[:, ["chemical formula", "Tc (of this sample) recommended", "oxygen", "common formula of oxygen", "measured value of Oxygen content", "title of reference"]].rename(columns={"chemical formula": "formula", "Tc (of this sample) recommended": "Tc"})

In [22]:
crab_input_data = crab_input_data[crab_input_data["Tc"].notna()]
crab_input_data = crab_input_data.iloc[1:, :]

In [8]:
#現状判明していることとして、酸素数関連のデータに問題がいろいろとあるので
#酸素がある物質については酸素の具体的な量が乗っているデータは組成式にそれを組み込んで
#使ってないデータは消す方針でいったん動く
#まず酸化物とそうでないものを分ける

In [23]:
NonOxide_SC_data = crab_input_data[crab_input_data["oxygen"].isna()].drop(columns=["oxygen", "common formula of oxygen", "measured value of Oxygen content"])
Oxide_SC_data = crab_input_data[crab_input_data["oxygen"].notna()]

In [24]:
#measured valueがある場合
measured_value_of_Oxygen_index = Oxide_SC_data.loc[:, "measured value of Oxygen content"].notna()
Oxide_SC_data.loc[measured_value_of_Oxygen_index, "value of oxygen"] = Oxide_SC_data.loc[measured_value_of_Oxygen_index, "measured value of Oxygen content"]


In [25]:
#measured valueがないがcommon formulaに数字がある場合
common_value_of_Oxygen_index = Oxide_SC_data.loc[:, "measured value of Oxygen content"].isna() & Oxide_SC_data.loc[:, "common formula of oxygen"].str.isdigit()
Oxide_SC_data.loc[common_value_of_Oxygen_index, "value of oxygen"] = Oxide_SC_data.loc[common_value_of_Oxygen_index, "common formula of oxygen"]

In [26]:
final_Oxide_SC_data = Oxide_SC_data.drop(columns=["oxygen", "common formula of oxygen", "measured value of Oxygen content"])
final_Oxide_SC_data = final_Oxide_SC_data[final_Oxide_SC_data["value of oxygen"].notna()]

In [27]:
new_SC_data = pd.concat([final_Oxide_SC_data, NonOxide_SC_data], axis = 0).drop(columns=["value of oxygen"]).reset_index(drop = True)

In [28]:
new_SC_data["Tc"] = new_SC_data["Tc"].astype(float)

In [29]:
pd.set_option('display.max_colwidth', 500)

In [12]:
display(final_SC_data.head(50))

NameError: name 'final_SC_data' is not defined

In [30]:
new_SC_data = new_SC_data.sort_values("Tc", ascending = False)

In [31]:
check_words = ["pressure", "gpa", "mbar"]

In [32]:
high_pressure_data = new_SC_data["title of reference"].astype(str).apply(lambda x: any(word.lower() in x.lower() for word in check_words))
not_high_pressure_data = ~high_pressure_data

In [33]:
final_SC_data = new_SC_data[not_high_pressure_data].iloc[5:, :].drop(columns = ["title of reference"])

In [34]:
final_SC_data.to_csv("/workspaces/mat2vec_codespace/main_code/final_SC_X_data.csv", index = False)